# How Do Mice's Food-interaction Dynamic Changes according to their Hunger Levels and the Value of Food Given?

Experiment:
- mice: 6-hr or 24-hr fasted
- Open Field Test: mice were place individually in an open arena with a food source placed in the center, they were allowed to interact with it undistrubed in the dark for ~ 20 mins
- food source given: calorie-densed gel or low-calorie gel

In [3]:
import pandas as pd ; import numpy as np; import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import coordinates ; import lc_functions

#### Importing Data

In [5]:
#From dlc
dlc_all = pd.read_csv("F0C3_24_cam_top_DLC_resnet50_OF_all_8_04Apr8shuffle1_30000.csv",header = 1)
dlc = dlc_all.drop([0])

nose = np.array(list(zip(dlc['Nose'], dlc['Nose.1'])))[300:]  #subtracting initial 300 df - artifact
cm_to_pix = 430/40 # ratio pix to cm

nose_cm = np.zeros(np.shape(nose))
for i in range(len(nose)):
    nose_cm[i] = np.array([float(nose[i][n])/cm_to_pix for n in range(2)])

## Defining 3 Behavioral States that Describes Food Interaction in 1 Trial

To gauge the behavioral dynamics of each trial, I decided to segregate the mice's behavioral state in each trial into 3, then calculate the probability in which they transition between them.

3 Behavioral States
- 1) non-food interactions
- 2) short interaction with food (e.g. sniffing)
- 3) long interaction with food (e.g. eating/licking)

In [6]:
#computes '?' inside source coordinate
in_nose = np.zeros(nose[:,0].shape)
for i in range(len(nose)):
    in_nose[i] = np.array(lc_functions.inside_cor(nose_cm[i]))

In [7]:
def threebehaviors(threshold, in_nose):
    #long interaction - every inside source that  >= threshold
    long_int = lc_functions.dur_long_time(in_nose, threshold) 
    f_long = lc_functions.convert_np_to_wholelist(long_int)
    
    #short interaction - every inside source that < threshold
    short_int =  lc_functions.dur_short_time(in_nose, threshold) 
    f_short = lc_functions.convert_np_to_wholelist(short_int)
    
    #non-interaction
    f_interactions = set(f_long + f_short)

    all_f = set(np.arange(0,len(nose_cm)))
    f_nonfood = all_f  - all_f.intersection(f_interactions)
    
    return (f_long, f_short, list(f_nonfood))   

## Calculate the Transition Probability between the 3 Behavior States

In [9]:
def transition_count(threshold_for_long, in_nose):
    f_long, f_short, f_nonfood = threebehaviors(threshold_for_long, in_nose)
    all_f = np.arange(0,len(nose_cm))
    all_f[f_nonfood] = 0 #non-food interaction
    all_f[f_long] = 1 #long food interaction
    all_f[f_short] = 2 #short food interaction
    
    chunked_all_f = all_f[np.insert(np.diff(all_f).astype(np.bool), 0, True)]
    
    #building transition matrix
    trans_dict  = {"long_short":[],"long_non":[],"short_long":[] 
                ,"short_non":[],"non_short":[],"non_long":[]};
    #counting
    c_long_short = 0 ; c_long_non = 0 ; c_short_long = 0
    c_short_non = 0 ; c_non_short = 0; c_non_long = 0 

    for i in range(len(chunked_all_f)-1):
        if chunked_all_f[i] == 1:
            if chunked_all_f[i+1] == 2:
                c_long_short +=1
            elif chunked_all_f[i+1] == 0:
                c_long_non +=1
        elif chunked_all_f[i] == 2:
            if chunked_all_f[i+1] == 1:
                c_short_long +=1
            elif chunked_all_f[i+1] == 0:
                c_short_non +=1
        elif chunked_all_f[i] == 0:
            if chunked_all_f[i+1] == 2:
                c_non_short +=1
            elif chunked_all_f[i+1] == 1:
                c_non_long +=1
        
    #append back in dictionary
    trans_dict["long_short"].append(c_long_short)
    trans_dict["long_non"].append(c_long_non)
    trans_dict["short_long"].append(c_short_long)
    trans_dict["short_non"].append(c_short_non)
    trans_dict["non_short"].append(c_non_short)
    trans_dict["non_long"].append(c_non_long)

    return trans_dict

In [28]:
def trans_matrix(threshold_for_long, in_nose):
    t_dict = transition_count(threshold_for_long, in_nose)
    total_trans = sum(t_dict[item][0] for item in t_dict)

    p_long_non = t_dict['long_non'][0]/total_trans
    p_short_non = t_dict['short_non'][0]/total_trans
    p_non_short = t_dict['non_short'][0]/total_trans
    p_non_long = t_dict['non_long'][0]/total_trans
    p_long_long = 1 - p_long_non
    p_short_short = 1 - p_short_non
    p_non_non = 1 - p_non_short - p_non_long
    
    m = np.array([
        [p_non_non, p_long_non, p_short_non ],
        [p_non_long, p_long_long, 0],
        [p_non_short, 0, p_short_short]],dtype=object)
    
    row_labels = ['Non-food int', 'long food int', 'short food int']
    column_labels = ['Non-food int', 'long food int', 'short food int']
    
    import pandas
    df = pandas.DataFrame(m, columns=column_labels, index=row_labels)
    return df

In [29]:
threshold_for_long = 100 #time frame
print('time for long interaction = ' + str(threshold_for_long/33) + 's')
print(trans_matrix(threshold_for_long, in_nose))

time for long interaction = 3.0303030303030303s
               Non-food int long food int short food int
Non-food int            0.5     0.0820896        0.41791
long food int     0.0820896       0.91791              0
short food int      0.41791             0        0.58209


/Users/alisonsin/Desktop/thesis/lc_functions.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  split_ind_one = np.array(np.split(ind_one, np.where(np.diff(ind_one) != 1)[0]+1))
/Users/alisonsin/Desktop/thesis/lc_functions.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  split_ind_one = np.array(np.split(ind_one, np.where(np.diff(ind_one) != 1)[0]+1))
/Users/alisonsin/Desktop/thesis/lc_functions.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different